In [1]:
# import the library 
import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Model, layers, datasets
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import random
import numpy as np
from scipy import ndimage as ndi
from skimage.util import random_noise
from skimage import feature
import rawpy
import ipywidgets as widgets
from IPython.display import display

# Marching cube 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure
from skimage.draw import ellipsoid

from fringe.utils.io import import_image, export_image
from fringe.utils.modifiers import ImageToArray, ConvertToTensor
from fringe.process.gpu import AngularSpectrumSolver as AsSolver

def dng_to_gray(FILE_PATH):
    with rawpy.imread(FILE_PATH) as raw:
    # Convert the raw image to a RGB array
        rgb = raw.postprocess()
    gray_image = 0.25 * rgb[:, :, 0] + 0.250 * rgb[:, :, 1] + 0.50 * rgb[:, :, 2]
    return gray_image

def im2double(im):
    min_val = np.min(im.ravel())
    max_val = np.max(im.ravel())
    out = (im.astype('float') - min_val) / (max_val - min_val)
    return out

def Scale(img, perc, max_val):
	img *= perc
	img += 1 - perc
	img /= max_val
	return img

def generate_stack (hologram, z_layers):
    holo_list = []
    for i in z_layers:
        hologram_amp = tf.math.abs(hologram)
        solver = AsSolver(shape=hologram_amp.shape, dx=1.12, dy=1.12, wavelength=650e-3)
        
        # Adjusting contrast
        hologram_amp = Scale(hologram_amp, perc=1, max_val=np.max(hologram_amp))

        rec = solver.solve(hologram, i)
        amp = np.abs(rec)
        holo_list.append(amp)
    stacked_holo = np.stack([holo_list], axis = 0)
    return stacked_holo[0]

def display_image(IMAGE_STACK, index, z_layers):
    plt.title(f"Image at distance: {z_layers[index]} um ")
    plt.imshow(IMAGE_STACK[index], cmap='gray')
    plt.axis('off')
    plt.colorbar()
    plt.show()


2025-03-24 15:33:16.743085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/chuckles/holoVirtualEnv/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import os

# Specify the directory containing the files
directory = '/Users/chuckles/holoVirtualEnv/holoimg/photo79656.jpg'
save_path = '/Users/chuckles/holoVirtualEnv/holoimg/'
# Iterate through the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.dng'):
        file_path = os.path.join(directory, filename)
        print(f'Processing file: {file_path}')
        FILE_PATH = file_path
        hologram=im2double(dng_to_gray(FILE_PATH))
        z_layers = range (2800, 2820, 20) 
        reconstruction_stack = generate_stack(hologram, z_layers)
        # Change the filename to .png
        new_filename = os.path.splitext(filename)[0] + '.png'
        save_name = os.path.join(save_path, new_filename)
        plt.imsave(save_name, reconstruction_stack[0])


Processing file: /Users/chuckles/holoVirtualEnv/holoimg/03-06-112541-target_closer.dng


NotADirectoryError: [Errno 20] Not a directory: '/Users/chuckles/holoVirtualEnv/holoimg/photo79656.jpg/03-06-112541-target_closer.png'

In [2]:
def rgb565_to_rgb888(data):
    # Unpack the RGB565 data into a numpy array
    rgb565 = np.frombuffer(data, dtype=np.uint16)
    
    # Extract the RGB components
    r = (rgb565 >> 11) & 0x1F
    g = (rgb565 >> 5) & 0x3F
    b = rgb565 & 0x1F
    
    # Convert to 8-bit values
    r = (r * 255 // 31).astype(np.uint8)
    g = (g * 255 // 63).astype(np.uint8)
    b = (b * 255 // 31).astype(np.uint8)
    
    # Stack the RGB channels together
    rgb888 = np.stack((r, g, b), axis=-1)
    
    return rgb888

def read_rgb565_file(file_path, width, height):
    with open(file_path, 'rb') as f:
        data = f.read()
    
    # Convert RGB565 to RGB888
    rgb_image = rgb565_to_rgb888(data)
    
    # Create an image using PIL
    image = Image.fromarray(rgb_image, 'RGB')
    
    return image

# Example usage
file_path = '/Users/chuckles/holoVirtualEnv/holoimg/photo51766'
width = 320  # Set the width of the image
height = 240  # Set the height of the image

image = read_rgb565_file(file_path, width, height)
image.show()  # Display the image
# image.save('output_image.png')  # Optionally save the image

NameError: name 'Image' is not defined

In [ ]:
# Specify the directory containing the files
directory = '/Users/chuckles/holoVirtualEnv/holoimg/photo79656.jpg'

directory = "/Users/chuckles/holoVirtualEnv/holoimg/photo51766"

# Iterate through the files in the directory
file_path = directory
print(f'Processing file: {file_path}')
FILE_PATH = file_path

p1 = ImageToArray(bit_depth=16, channel='gray', crop_window=None, dtype='float32')
hologram = import_image(file_path, modifiers=[p1])

z_layers = range (3000, 3200, 10) 
reconstruction_stack = generate_stack(hologram, z_layers)
# Change the filename to .png
# new_filename = os.path.splitext(filename)[0] + '.png'
# save_name = os.path.join(save_path, new_filename)
# plt.imsave(save_name, reconstruction_stack[0])


Processing file: /Users/chuckles/holoVirtualEnv/holoimg/photo79656.jpg


In [ ]:
images = reconstruction_stack
# images_threshold = stacked_holo[0]
num_images = np.shape(images)[0]


# Create a function to display the selected image
def display_image(index):
    # # Apply soft thresholding
    # threshold = 0.50
    # range_width = 0.40
    # images_threshold[index] = (soft_threshold(images[index], threshold, range_width))

    # Update the title based on the current index (z_layer)
    plt.figure(figsize=(15, 15))  # Set the figure size (width, height) in inches
    plt.imshow(images[index], cmap='gray')
    plt.title(f"Image at distance: {z_layers[index]} um ")
    plt.axis('off')
    plt.colorbar(orientation='vertical')  # You can change orientation to 'horizontal' if needed
    plt.show()

def save_image(index):
    plt.imsave(f'saved_image_{index}.jpg', images[index], cmap='gray')
    print(f'Image saved as saved_image_{index}.jpg')

# Create a slider for selecting the image index
image_slider = widgets.IntSlider(value=0, min=0, max=len(images)-1, step=1, description='Image Index')

# Create a button to save the image
save_button = widgets.Button(description="Save Image")

# Link the button click event to the save_image function
def on_save_button_click(b):
    save_image(image_slider.value)

save_button.on_click(on_save_button_click)

# Create an interactive widget
interactive_display = widgets.interactive(display_image, index=image_slider)

# Display the slider and the save button
display(save_button)
display(interactive_display)


Button(description='Save Image', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='Image Index', max=19), Output()), _dom_classes=('widget-…

In [ ]:
import numpy as np
from PIL import Image

# Create a stack of random images (for demonstration purposes)
num_images = 10  # Number of images in the stack
image_height = 100
image_width = 100

# Create a list to hold the images
images = []

for i in range(num_images):
    # Create a random image
    random_image = np.random.rand(image_height, image_width, 3) * 255  # Random RGB image
    random_image = random_image.astype('uint8')  # Convert to uint8
    images.append(Image.fromarray(random_image))

# Specify the filename for the GIF
gif_filename = 'output_animation.gif'

# Save the images as a GIF
images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=100, loop=0)

print(f'GIF saved as: {gif_filename}')